In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [12]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [13]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [14]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

**Part 1 - Neural Network**

In [15]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

In [17]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 39ms/step - loss: 35.1919 - root_mean_squared_error: 5.9323 - val_loss: 34.7154 - val_root_mean_squared_error: 5.8920
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 33.0942 - root_mean_squared_error: 5.7528 - val_loss: 32.3982 - val_root_mean_squared_error: 5.6919
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 31.3194 - root_mean_squared_error: 5.5964 - val_loss: 30.2379 - val_root_mean_squared_error: 5.4989
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 29.6246 - root_mean_squared_error: 5.4429 - val_loss: 28.2147 - val_root_mean_squared_error: 5.3118
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 27.9859 - root_mean_squared_error: 5.2902 - val_loss: 26.3160 - val_root_mean_squared_error: 5.1299
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 26.3999 - root_mean_squared_error

In [18]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.7 - Actual: 7.0
Predicted: 5.3 - Actual: 5.0
Predicted: 5.4 - Actual: 5.0
Predicted: 6.5 - Actual: 5.0
Predicted: 4.9 - Actual: 5.0
Predicted: 6.6 - Actual: 5.0
Predicted: 6.0 - Actual: 7.0
Predicted: 6.4 - Actual: 7.0
Predicted: 6.4 - Actual: 7.0
Predicted: 6.2 - Actual: 5.0


**Part 2 - BNN**

In [19]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [20]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [21]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 178ms/step - loss: 36.5009 - root_mean_squared_error: 6.0404 - val_loss: 34.0829 - val_root_mean_squared_error: 5.8368
Epoch 2/500
5/5 [==============================] - 0s 22ms/step - loss: 33.7498 - root_mean_squared_error: 5.8082 - val_loss: 34.8041 - val_root_mean_squared_error: 5.8982
Epoch 3/500
5/5 [==============================] - 0s 21ms/step - loss: 33.3096 - root_mean_squared_error: 5.7700 - val_loss: 33.1519 - val_root_mean_squared_error: 5.7564
Epoch 4/500
5/5 [==============================] - 0s 21ms/step - loss: 35.8263 - root_mean_squared_error: 5.9843 - val_loss: 28.2532 - val_root_mean_squared_error: 5.3138
Epoch 5/500
5/5 [==============================] - 0s 22ms/step - loss: 32.1786 - root_mean_squared_error: 5.6713 - val_loss: 33.3899 - val_root_mean_squared_error: 5.7772
Epoch 6/500
5/5 [==============================] - 0s 26ms/step - loss: 34.6254 - root_mean_squared_error: 5.88

In [22]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.62, min: 5.0, max: 5.99, range: 1.0 - Actual: 7.0
Predictions mean: 5.17, min: 4.23, max: 5.74, range: 1.51 - Actual: 5.0
Predictions mean: 5.27, min: 4.74, max: 5.85, range: 1.11 - Actual: 5.0
Predictions mean: 6.29, min: 5.91, max: 6.46, range: 0.55 - Actual: 5.0
Predictions mean: 5.13, min: 4.3, max: 5.86, range: 1.57 - Actual: 5.0
Predictions mean: 6.35, min: 6.04, max: 6.48, range: 0.44 - Actual: 5.0
Predictions mean: 6.14, min: 5.84, max: 6.42, range: 0.58 - Actual: 7.0
Predictions mean: 6.23, min: 5.88, max: 6.42, range: 0.55 - Actual: 7.0
Predictions mean: 6.32, min: 6.04, max: 6.49, range: 0.45 - Actual: 7.0
Predictions mean: 6.19, min: 5.78, max: 6.44, range: 0.66 - Actual: 5.0


In [23]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 44ms/step - loss: 42.8731 - root_mean_squared_error: 6.5474 - val_loss: 42.7638 - val_root_mean_squared_error: 6.5391
Epoch 2/500
17/17 [==============================] - 0s 8ms/step - loss: 39.3609 - root_mean_squared_error: 6.2735 - val_loss: 39.5413 - val_root_mean_squared_error: 6.2878
Epoch 3/500
17/17 [==============================] - 0s 8ms/step - loss: 39.6939 - root_mean_squared_error: 6.3000 - val_loss: 35.9705 - val_root_mean_squared_error: 5.9972
Epoch 4/500
17/17 [==============================] - 0s 8ms/step - loss: 36.4771 - root_mean_squared_error: 6.0392 - val_loss: 35.2591 - val_root_mean_squared_error: 5.9375
Epoch 5/500
17/17 [==============================] - 0s 8ms/step - loss: 35.0113 - root_mean_squared_error: 5.9167 - val_loss: 37.0432 - val_root_mean_squared_error: 6.0858
Epoch 6/500
17/17 [==============================] - 0s 8ms/step - loss: 33.7584 - root_mean_squared_error

**Part 3 - Probabilistic Bayesian neural network**

In [24]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [25]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 48ms/step - loss: 34.0081 - root_mean_squared_error: 6.5024 - val_loss: 51.4240 - val_root_mean_squared_error: 6.6757
Epoch 2/1000
17/17 [==============================] - 0s 8ms/step - loss: 26.4744 - root_mean_squared_error: 6.3981 - val_loss: 32.7554 - val_root_mean_squared_error: 6.4915
Epoch 3/1000
17/17 [==============================] - 0s 8ms/step - loss: 26.8944 - root_mean_squared_error: 6.3582 - val_loss: 24.8716 - val_root_mean_squared_error: 6.1750
Epoch 4/1000
17/17 [==============================] - 0s 8ms/step - loss: 22.9882 - root_mean_squared_error: 6.3393 - val_loss: 16.1172 - val_root_mean_squared_error: 6.3259
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 16.1383 - root_mean_squared_error: 6.2340 - val_loss: 19.5036 - val_root_mean_squared_error: 6.4070
Epoch 6/1000
17/17 [==============================] - 0s 8ms/step - loss: 16.3891 - root_mean_squared

In [26]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.56, stddev: 0.7, 95% CI: [6.93 - 4.19] - Actual: 7.0
Prediction mean: 5.07, stddev: 0.67, 95% CI: [6.38 - 3.76] - Actual: 5.0
Prediction mean: 5.15, stddev: 0.67, 95% CI: [6.46 - 3.83] - Actual: 5.0
Prediction mean: 6.5, stddev: 0.87, 95% CI: [8.21 - 4.79] - Actual: 5.0
Prediction mean: 5.01, stddev: 0.66, 95% CI: [6.31 - 3.71] - Actual: 5.0
Prediction mean: 6.57, stddev: 0.87, 95% CI: [8.28 - 4.87] - Actual: 5.0
Prediction mean: 6.2, stddev: 0.77, 95% CI: [7.71 - 4.68] - Actual: 7.0
Prediction mean: 6.46, stddev: 0.87, 95% CI: [8.17 - 4.74] - Actual: 7.0
Prediction mean: 6.56, stddev: 0.85, 95% CI: [8.22 - 4.9] - Actual: 7.0
Prediction mean: 6.26, stddev: 0.8, 95% CI: [7.84 - 4.69] - Actual: 5.0
